In [10]:
# import dependancies
import pandas as pd
import json
import os

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean
from sqlalchemy.orm import relationship

In [2]:
# Create classes
class RestaurantCls(Base):
    __tablename__ = 'Restaurant'
    id = Column(Integer, primary_key=True)
    yelp_id = Column(String(255))
    name = Column(String(255))
    # location
    state = Column(String(2)) 
    city = Column(String(100))
    street = Column(String(255))
    zipcode = Column(String(35))
    # Operational info
    price = Column(Integer)
    rating = Column(Float)
    isclosed = Column(Boolean)
    
class CategoryCls(Base):
    __tablename__ = 'Category'
    id = Column(Integer, primary_key=True)
    alias = Column(String(255))
    title = Column(String(255))
    

In [3]:
# read data from files created by calling Yelp API
#data_file=pd.read_json("data.txt", lines=False)
#data_file
#data = json.load(open('data.txt'))
#df = pd.DataFrame(data["businesses"])
#df

In [4]:
#data = json.load(open('YelpData.txt'))
#df = pd.DataFrame(data["businesses"])
#df

### Load data from file into objects for further processing
* Use direct data reader, not Pandas df to get values


In [5]:

def read_rest_data(file):

    with open(file, "r") as read_file:
        restaurants = json.load(read_file)["businesses"]

    # Hash of categories
    categories = {}

    # List of restaurants
    rest_list = []

    # List of restaurant categories
    rest_cat = []
    
    for restaurant in restaurants:
        id = restaurant["id"]
        name = restaurant["name"]

        # location
        location = restaurant["location"]
        # All address entries into street
        street = location["address1"]
        street2 = location["address2"]
        street3 = location["address3"]
        if(street2 and not street2.isspace()):
            street += " " + street2 
        if(street3 and not street3.isspace()):
            street += " " + street3 

        # operational info
        for c in restaurant["categories"]:
            # Object to list of categories 
            rest_cat.append((id, CategoryCls(alias = c['alias'], title = c['title'] )))
            categories[c['alias']] = c['title']

        if(restaurant.get("price")):
            price = restaurant["price"].count('$') 
        else:
            price = None # some entries miss price

        restaurantObj = RestaurantCls(
            yelp_id = restaurant["id"],
            name = restaurant["name"],
            city = location["city"],
            street = street,
            zipcode = location["zip_code"],
            price = price,
            rating = float(restaurant["rating"]),
            isclosed = bool(restaurant["is_closed"])
        )
        rest_list.append(restaurantObj)
    
    print(len(rest_list))
    return rest_list, rest_cat, categories


### Load data from file into objects for further processing


In [6]:
fn = os.path.join("..", "Data", "YelpData.txt")
restaurants, restaurant_categories, categories = read_rest_data(fn)

50


In [7]:
# Load categories into database
categories

{'icecream': 'Ice Cream & Frozen Yogurt',
 'bakeries': 'Bakeries',
 'french': 'French',
 'soulfood': 'Soul Food',
 'breakfast_brunch': 'Breakfast & Brunch',
 'newamerican': 'American (New)',
 'wine_bars': 'Wine Bars',
 'cafes': 'Cafes',
 'desserts': 'Desserts',
 'seafood': 'Seafood',
 'seafoodmarkets': 'Seafood Markets',
 'raw_food': 'Live/Raw Food',
 'customcakes': 'Custom Cakes',
 'tradamerican': 'American (Traditional)',
 'steak': 'Steakhouses',
 'burmese': 'Burmese',
 'asianfusion': 'Asian Fusion',
 'greek': 'Greek',
 'mediterranean': 'Mediterranean',
 'chinese': 'Chinese',
 'chicken_wings': 'Chicken Wings',
 'noodles': 'Noodles',
 'modern_european': 'Modern European',
 'cakeshop': 'Patisserie/Cake Shop',
 'macarons': 'Macarons',
 'mexican': 'Mexican',
 'bars': 'Bars',
 'vietnamese': 'Vietnamese',
 'sandwiches': 'Sandwiches',
 'pizza': 'Pizza',
 'italian': 'Italian',
 'cocktailbars': 'Cocktail Bars',
 'ramen': 'Ramen',
 'british': 'British',
 'fishnchips': 'Fish & Chips',
 'donuts'

In [8]:
# Load restaurants into database
for r in restaurants:
    print(r.name)


Bi-Rite Creamery
Brenda's French Soul Food
Gary Danko
Tartine Bakery & Cafe
Hog Island Oyster Co
Mitchell's Ice Cream
House of Prime Rib
Burma Superstar
The House
Kokkari Estiatorio
San Tung
Nopa
B Patisserie
El Farolito
Fog Harbor Fish House
Zazie
Saigon Sandwich
Golden Boy Pizza
Sweet Maple
Liholiho Yacht Club
Tony's Pizza Napoletana
Sotto Mare Oysteria & Seafood
Foreign Cinema
Golden Gate Bakery
Little Star Pizza
Marufuku Ramen SF
Mama's On Washington Square
Chapeau!
Humphry Slocombe Ice Cream
The Codmother Fish & Chips
La Taqueria
Bob's Donut & Pastry Shop
Golden Gate Bridge
Twin Peaks
State Bird Provisions
Arizmendi Bakery
Espetus Brazilian Steak House
Mission Dolores Park
Dottie's True Blue Cafe
Kitchen Story
Suppenküche
Beretta
Philz Coffee
Schubert's Bakery
Alcatraz Island
Golden Gate Park
HRD
Anthony's Cookies
Arsicault Bakery
Swan Oyster Depot


In [9]:
# load resturant categories into database
for r in restaurant_categories:
    print(f"{r[0]}; {r[1].alias}; {r[1].title}")

wGl_DyNxSv8KUtYgiuLhmA; icecream; Ice Cream & Frozen Yogurt
wGl_DyNxSv8KUtYgiuLhmA; bakeries; Bakeries
lJAGnYzku5zSaLnQ_T6_GQ; french; French
lJAGnYzku5zSaLnQ_T6_GQ; soulfood; Soul Food
lJAGnYzku5zSaLnQ_T6_GQ; breakfast_brunch; Breakfast & Brunch
WavvLdfdP6g8aZTtbBQHTw; newamerican; American (New)
WavvLdfdP6g8aZTtbBQHTw; french; French
WavvLdfdP6g8aZTtbBQHTw; wine_bars; Wine Bars
ri7UUYmx21AgSpRsf4-9QA; bakeries; Bakeries
ri7UUYmx21AgSpRsf4-9QA; cafes; Cafes
ri7UUYmx21AgSpRsf4-9QA; desserts; Desserts
Xg-FyjVKAN70LO4u4Z1ozg; seafood; Seafood
Xg-FyjVKAN70LO4u4Z1ozg; seafoodmarkets; Seafood Markets
Xg-FyjVKAN70LO4u4Z1ozg; raw_food; Live/Raw Food
76smcUUGRvq3k1MVPUXbnA; icecream; Ice Cream & Frozen Yogurt
76smcUUGRvq3k1MVPUXbnA; customcakes; Custom Cakes
oT08T3Vpn1I7jDmrBBRMTw; tradamerican; American (Traditional)
oT08T3Vpn1I7jDmrBBRMTw; steak; Steakhouses
oT08T3Vpn1I7jDmrBBRMTw; wine_bars; Wine Bars
eYXwVR4mMAjzkJnm5wneHQ; burmese; Burmese
ZoZjbOYR-apY8XvommlNUA; asianfusion; Asian Fusion